<a href="https://colab.research.google.com/github/dkurbatovv/Python/blob/main/Sign_Language_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

In [ ]:
train = pd.read_csv('sign_mnist_train.csv')
test = pd.read_csv('sign_mnist_test.csv')

In [ ]:
train.head()

In [ ]:
train.label.value_counts()

In [ ]:
X = train.drop('label', axis = 1).to_numpy()
y = train['label'].to_numpy()

In [ ]:
g = sns.countplot(y)

In [ ]:
y = to_categorical(y, num_classes=25)

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state=2)


In [ ]:
X_train = X_train.reshape(-1,28,28,1)
X_val = X_val.reshape(-1,28,28,1)

In [ ]:
X_train = X_train.astype(float) / 255.0
X_val = X_val.astype(float) / 255.0

In [ ]:
g = plt.imshow(X_train[100][:,:,0])

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(filters = 32, kernel_size = (5,5), padding = 'same', activation = 'relu', input_shape=(28,28, 1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5), padding = 'same', activation = 'relu'))

model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters = 64, kernel_size = (5,5), padding = 'same', activation = 'relu'))
model.add(Conv2D(filters = 64, kernel_size = (5,5), padding = 'same', activation = 'relu'))

model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(25, activation = 'softmax'))


In [ ]:
optimizer = RMSprop(lr = 0.001, rho = 0.9, epsilon = 1e-08, decay = 0.0)

In [ ]:
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
#отжиг скорости обучения
learning_rate_reduction = ReduceLROnPlateau(monitor="val_loss", 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
history = model.fit(X_train, y_train, batch_size = 64, epochs = 20, validation_data = (X_val, y_val), verbose = 2)

In [ ]:
plt.plot(history.history['accuracy'], label = 'accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

In [ ]:
plt.plot(history.history['loss'], label = 'loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

In [ ]:
results = model.evaluate(X_val, y_val, batch_size = 64)
print("test loss, test acc:", results)

In [ ]:
X_test_value = test.drop('label', axis = 1).to_numpy()

In [ ]:
y_test_value = test['label']

In [ ]:
y_test_value = to_categorical(y_test_value, num_classes = 25)

In [ ]:
X_test_value = X_test_value.reshape(-1, 28, 28, 1)
X_test_value = X_test_value.astype(float) / 255.0

In [ ]:
y_pred = model.predict(X_test_value)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test_value, y_pred.round())